<a href="https://colab.research.google.com/github/ekaratnida/Automated_Tools-/blob/keep/Mini-bads3/Module3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [13]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [1]:
import pandas as pd
from google.colab import auth
import os
import pandas as pd
import sqlite3 as sql

auth.authenticate_user()
os.chdir('/content/dataset/')

FileNotFoundError: ignored

In [ ]:
df = pd.read_csv('Supermarket.csv')
df.head()
conn = sql.connect("supermarket.db")
df.to_sql("table1", conn, if_exists='replace')

In [ ]:
sql_string = "select * from table1"
df = pd.read_sql(sql_string, conn)
df.head()

In [ ]:
sql ='''
SELECT CUST_CODE, 
COUNT(DISTINCT BASKET_ID)/COUNT(DISTINCT SHOP_WEEK) AS VISIT_PER_WEEK,
COUNT(DISTINCT SHOP_WEEK) AS TOTAL_WEEK,
COUNT(DISTINCT PROD_CODE) AS TOTAL_PRODUCT,
SUM(SPEND) AS TOTAL_SALES
FROM table1
WHERE CUST_CODE IS NOT NULL GROUP BY CUST_CODE
HAVING TOTAL_SALES < 4000
'''

df = pd.read_sql(sql, conn)
print(df[0:3])

In [ ]:
sql2 = '''select * from table1 where cust_code = "CUST0000003603"'''
df2 = pd.read_sql(sql2, conn)
df2

In [16]:
#https://drive.google.com/file/d/1tV3wDsawGcH-XjWXfWnHWGeNbf0Tp46q/view?usp=sharing
fileDownloaded = drive.CreateFile({'id':'1tV3wDsawGcH-XjWXfWnHWGeNbf0Tp46q'})


In [17]:
fileDownloaded.GetContentFile('Supermarket.csv')

In [18]:
import pandas as pd
df = pd.read_csv('Supermarket.csv')

df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200812,20080516,6,9,1,0.01,PRD0902611,CL00241,DEP00086,G00028,D00008,CUST0000767429,XX,OT,994111000667718,S,MM,Small Shop,Grocery,STORE01026,SS,N01
1,200646,20070108,2,16,1,1.61,PRD0901483,CL00236,DEP00084,G00028,D00008,CUST0000419057,XX,PE,994104000413959,M,UM,Top Up,Fresh,STORE01300,LS,S03
2,200812,20080518,1,15,3,0.03,PRD0900841,CL00212,DEP00070,G00022,D00005,CUST0000714492,XX,OA,994111000633261,M,LA,Small Shop,Nonfood,STORE00343,XLS,W02
3,200632,20061005,5,10,1,1.11,PRD0904263,CL00248,DEP00089,G00030,D00009,CUST0000356307,XX,OA,994102600389888,S,MM,Small Shop,Grocery,STORE01007,XLS,W01
4,200707,20070415,1,8,23,0.23,PRD0902611,CL00241,DEP00086,G00028,D00008,CUST0000058615,XX,OT,994105300205367,L,MM,Top Up,Grocery,STORE01794,LS,W01


In [ ]:
x = df.iloc[:,1:-1]
x.head(5)

In [ ]:
y = df.iloc[:,-1:]
y.head(5)

In [ ]:
import statsmodels.api as sm
x = sm.add_constant(x)
print(x)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
import statsmodels.api as sm

train_result = sm.OLS(y_train, x_train).fit()
train_result.summary()



In [ ]:
train_result.params

In [ ]:
train_result.predict([1, 1, 3, 50])

In [ ]:
#train_result.predict([1, 1, 3, 5])
y_pred = train_result.predict(x_test)

from sklearn.metrics import mean_squared_error, r2_score
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_train, y_train)
reg.fit(x_train, y_train)
print(reg.coef_)
print(reg.intercept_)
y_pred1 = reg.predict(x_test)
print('Mean squared error: %.2f'
  % mean_squared_error(y_test, y_pred1))

In [ ]:
import sys
import io
import matplotlib.pyplot as plt
import numpy as np

class DisplayLossCurve(object):
  def __init__(self, print_loss=False):
    self.print_loss = print_loss

  """Make sure the model verbose is set to 1"""
  def __enter__(self):
    self.old_stdout = sys.stdout
    sys.stdout = self.mystdout = io.StringIO()
  
  def __exit__(self, *args, **kwargs):
    sys.stdout = self.old_stdout
    loss_history = self.mystdout.getvalue()
    loss_list = []
    for line in loss_history.split('\n'):
      if(len(line.split("loss: ")) == 1):
        continue
      loss_list.append(float(line.split("loss: ")[-1]))
    plt.figure()
    plt.plot(np.arange(len(loss_list)), loss_list)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

    if self.print_loss:
      print("=============== Loss Array ===============")
      print(np.array(loss_list))
      
    return True

In [ ]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(verbose=1,max_iter=3000,tol=0.00001)
with DisplayLossCurve():
  reg.fit(x_train, y_train)

y_pred2 = reg.predict(x_test)
print('Mean squared error: %.2f'
  % mean_squared_error(y_test, y_pred2))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#print(x_train.iloc[:,3])
#print(y_train.shape)
plt.figure(figsize=(20,7))
plt.scatter(x_train.iloc[:,3], y_train, color = 'red')
#plt.plot(x_train.iloc[:,1:2], np.reshape(train_result.predict([1, 2, 5, 50]),(-1,1)), color = 'blue')
plt.title('Supermarket')
plt.xlabel('Visit per week')
plt.ylabel('Total sales')
plt.show()